# Bitcoin Trading Agent - Data Collection

This notebook handles all data collection from multiple sources:
- Investing.com Bitcoin historical data (via Crawl4AI)
- CoinMarketCap API for current prices
- Yahoo Finance as backup
- Binance API for additional market data

## Observations Log
We will document key findings and observations after each step.

In [ ]:
# Import required libraries
import os
import sys
import pandas as pd
import numpy as np
import requests
import yfinance as yf
from datetime import datetime, timedelta
import asyncio
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Add src to path for imports
sys.path.append('../src')

print("✅ Libraries loaded successfully")
print(f"Environment variables loaded: {os.path.exists('../.env')}")

## 1. Crawl4AI Setup for Investing.com

Using Crawl4AI to scrape Bitcoin historical data from Investing.com with advanced features like JavaScript rendering and anti-detection.

In [ ]:
from crawl4ai import AsyncWebCrawler
from crawl4ai.extraction_strategy import LLMExtractionStrategy
import json

async def scrape_investing_btc_data():
    """
    Scrape Bitcoin historical data from Investing.com using Crawl4AI
    """
    url = "https://www.investing.com/crypto/bitcoin/historical-data"
    
    # Define extraction strategy using Groq with Llama 3.3 70B
    extraction_strategy = LLMExtractionStrategy(
        provider="groq",
        api_token=os.getenv('GROQ_API_KEY'),
        model="llama-3.3-70b-versatile",
        instruction="""
        Extract the historical Bitcoin price data from the table. 
        Return a JSON array with objects containing:
        - date: The date in YYYY-MM-DD format
        - price: The closing price as a number
        - open: The opening price as a number  
        - high: The highest price as a number
        - low: The lowest price as a number
        - volume: The volume as a number
        - change_pct: The percentage change as a number
        """
    )
    
    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(
            url=url,
            extraction_strategy=extraction_strategy,
            css_selector=".historical-data-table, table[data-test='historical-data-table']",
            wait_for="css:.historical-data-table",
            timeout=30000
        )
        
        return result

# Run the scraper
try:
    result = await scrape_investing_btc_data()
    
    if result.extracted_content:
        # Parse the JSON response
        btc_data = json.loads(result.extracted_content)
        investing_df = pd.DataFrame(btc_data)
        
        print(f"✅ Successfully scraped {len(investing_df)} records from Investing.com")
        print(f"Date range: {investing_df['date'].min()} to {investing_df['date'].max()}")
        print(investing_df.head())
        
        # Save to CSV
        investing_df.to_csv('../data/btc_investing_raw.csv', index=False)
        
    else:
        print("❌ No data extracted from Investing.com")
        
except Exception as e:
    print(f"❌ Error scraping Investing.com: {str(e)}")
    investing_df = None

### Observation 1: Investing.com Scraping Results

**Key Findings:**
- [ ] Document success rate of Crawl4AI scraping
- [ ] Note data quality and completeness
- [ ] Record any anti-bot detection issues
- [ ] Validate date ranges and data format

In [ ]:
# Create data directory if it doesn't exist
os.makedirs('../data', exist_ok=True)

# If scraping failed, create a fallback function
def fallback_investing_scraper():
    """
    Fallback method using requests + BeautifulSoup if Crawl4AI fails
    """
    try:
        from bs4 import BeautifulSoup
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        url = "https://www.investing.com/crypto/bitcoin/historical-data"
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find the historical data table
            table = soup.find('table', {'data-test': 'historical-data-table'})
            if not table:
                table = soup.find('table', class_='historical-data-table')
            
            if table:
                rows = table.find_all('tr')[1:]  # Skip header
                
                data = []
                for row in rows:
                    cols = row.find_all('td')
                    if len(cols) >= 6:
                        date_str = cols[0].text.strip()
                        price = cols[1].text.replace(',', '').replace('$', '').strip()
                        open_price = cols[2].text.replace(',', '').replace('$', '').strip()
                        high = cols[3].text.replace(',', '').replace('$', '').strip()
                        low = cols[4].text.replace(',', '').replace('$', '').strip()
                        volume = cols[5].text.replace(',', '').strip()
                        
                        data.append({
                            'date': date_str,
                            'price': float(price) if price else None,
                            'open': float(open_price) if open_price else None,
                            'high': float(high) if high else None,
                            'low': float(low) if low else None,
                            'volume': volume
                        })
                
                return pd.DataFrame(data)
            
    except Exception as e:
        print(f"Fallback scraper error: {str(e)}")
        return None
    
    return None

# Try fallback if primary method failed
if 'investing_df' not in locals() or investing_df is None:
    print("Trying fallback scraping method...")
    investing_df = fallback_investing_scraper()
    
    if investing_df is not None:
        print(f"✅ Fallback method successful: {len(investing_df)} records")
        investing_df.to_csv('../data/btc_investing_raw.csv', index=False)
    else:
        print("❌ Both scraping methods failed")

## 2. CoinMarketCap API Integration

Getting current Bitcoin price and market data from CoinMarketCap API for real-time updates.

In [ ]:
def get_coinmarketcap_data():
    """
    Fetch current Bitcoin data from CoinMarketCap API
    """
    api_key = os.getenv('COINMARKETCAP_API_KEY')
    
    if not api_key or api_key == 'your_coinmarketcap_api_key_here':
        print("⚠️ CoinMarketCap API key not configured")
        return None
    
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'
    parameters = {
        'symbol': 'BTC',
        'convert': 'USD'
    }
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key,
    }
    
    try:
        response = requests.get(url, headers=headers, params=parameters)
        data = response.json()
        
        if response.status_code == 200 and 'data' in data:
            btc_data = data['data']['BTC']
            quote = btc_data['quote']['USD']
            
            current_data = {
                'timestamp': datetime.now().isoformat(),
                'price': quote['price'],
                'volume_24h': quote['volume_24h'],
                'percent_change_1h': quote['percent_change_1h'],
                'percent_change_24h': quote['percent_change_24h'],
                'percent_change_7d': quote['percent_change_7d'],
                'market_cap': quote['market_cap'],
                'last_updated': quote['last_updated']
            }
            
            return current_data
        else:
            print(f"❌ CoinMarketCap API error: {data}")
            return None
            
    except Exception as e:
        print(f"❌ Error fetching CoinMarketCap data: {str(e)}")
        return None

# Fetch current data
cmc_data = get_coinmarketcap_data()

if cmc_data:
    print("✅ Current Bitcoin data from CoinMarketCap:")
    print(f"Price: ${cmc_data['price']:,.2f}")
    print(f"24h Change: {cmc_data['percent_change_24h']:.2f}%")
    print(f"Volume 24h: ${cmc_data['volume_24h']:,.0f}")
    print(f"Market Cap: ${cmc_data['market_cap']:,.0f}")
else:
    print("⚠️ CoinMarketCap data not available - using backup sources")

### Observation 2: CoinMarketCap API Performance

**Key Findings:**
- [ ] Note API response time and reliability
- [ ] Document rate limits encountered
- [ ] Compare price accuracy with other sources
- [ ] Record data freshness (last_updated)

## 3. Yahoo Finance Integration

Using yfinance as a reliable backup source for Bitcoin historical and current data.

In [ ]:
def get_yahoo_btc_data(period='1y'):
    """
    Fetch Bitcoin data from Yahoo Finance
    period options: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
    """
    try:
        btc_ticker = yf.Ticker("BTC-USD")
        hist_data = btc_ticker.history(period=period)
        
        if not hist_data.empty:
            # Reset index to get date as column
            hist_data = hist_data.reset_index()
            
            # Rename columns to match our standard format
            hist_data.columns = [col.lower() for col in hist_data.columns]
            hist_data['date'] = hist_data['date'].dt.strftime('%Y-%m-%d')
            hist_data['price'] = hist_data['close']
            
            # Get current info
            info = btc_ticker.info
            current_price = info.get('regularMarketPrice', hist_data['close'].iloc[-1])
            
            print(f"✅ Yahoo Finance data: {len(hist_data)} records")
            print(f"Date range: {hist_data['date'].iloc[0]} to {hist_data['date'].iloc[-1]}")
            print(f"Current price: ${current_price:,.2f}")
            print(f"Latest close: ${hist_data['close'].iloc[-1]:,.2f}")
            
            return hist_data, current_price
        else:
            print("❌ No data returned from Yahoo Finance")
            return None, None
            
    except Exception as e:
        print(f"❌ Error fetching Yahoo Finance data: {str(e)}")
        return None, None

# Fetch Yahoo Finance data
yahoo_df, yahoo_current = get_yahoo_btc_data(period='6mo')

if yahoo_df is not None:
    # Save Yahoo data
    yahoo_df.to_csv('../data/btc_yahoo_raw.csv', index=False)
    
    # Show recent data
    print("\nRecent data from Yahoo Finance:")
    print(yahoo_df[['date', 'open', 'high', 'low', 'close', 'volume']].tail())
else:
    print("⚠️ Yahoo Finance data not available")

### Observation 3: Yahoo Finance Data Quality

**Key Findings:**
- [ ] Compare data consistency with other sources
- [ ] Note any gaps or anomalies in historical data
- [ ] Document volume data accuracy
- [ ] Record API response reliability

## 4. Data Integration and Standardization

Combining data from all sources into a standardized format for analysis.

In [ ]:
def standardize_btc_data(*dataframes, source_names=None):
    """
    Standardize and combine Bitcoin data from multiple sources
    """
    if source_names is None:
        source_names = [f'source_{i}' for i in range(len(dataframes))]
    
    combined_data = []
    
    for df, source in zip(dataframes, source_names):
        if df is not None and not df.empty:
            # Create a copy to avoid modifying original
            df_copy = df.copy()
            
            # Ensure we have required columns
            required_cols = ['date', 'open', 'high', 'low', 'close', 'volume']
            
            # Map common column variations
            if 'price' in df_copy.columns and 'close' not in df_copy.columns:
                df_copy['close'] = df_copy['price']
            
            # Convert date to datetime if it's not already
            if 'date' in df_copy.columns:
                df_copy['date'] = pd.to_datetime(df_copy['date'])
            
            # Add source column
            df_copy['source'] = source
            
            # Select and reorder columns
            available_cols = [col for col in required_cols if col in df_copy.columns]
            df_final = df_copy[available_cols + ['source']].copy()
            
            combined_data.append(df_final)
            print(f"✅ Processed {len(df_final)} records from {source}")
        else:
            print(f"⚠️ No data available from {source}")
    
    if combined_data:
        # Combine all dataframes
        final_df = pd.concat(combined_data, ignore_index=True)
        
        # Remove duplicates based on date and source
        final_df = final_df.drop_duplicates(subset=['date', 'source'])
        
        # Sort by date
        final_df = final_df.sort_values('date').reset_index(drop=True)
        
        return final_df
    else:
        return None

# Combine available data sources
available_dfs = []
source_names = []

if 'investing_df' in locals() and investing_df is not None:
    available_dfs.append(investing_df)
    source_names.append('investing')

if 'yahoo_df' in locals() and yahoo_df is not None:
    available_dfs.append(yahoo_df)
    source_names.append('yahoo')

# Standardize and combine data
if available_dfs:
    combined_btc_data = standardize_btc_data(*available_dfs, source_names=source_names)
    
    if combined_btc_data is not None:
        print(f"\n✅ Combined dataset: {len(combined_btc_data)} total records")
        print(f"Date range: {combined_btc_data['date'].min()} to {combined_btc_data['date'].max()}")
        print(f"Sources: {combined_btc_data['source'].unique()}")
        
        # Save combined data
        combined_btc_data.to_csv('../data/btc_combined_raw.csv', index=False)
        
        # Show data summary
        print("\nData summary by source:")
        print(combined_btc_data.groupby('source').agg({
            'date': ['count', 'min', 'max'],
            'close': ['mean', 'min', 'max']
        }).round(2))
    else:
        print("❌ Failed to combine data sources")
else:
    print("❌ No data sources available")

### Observation 4: Data Integration Results

**Key Findings:**
- [ ] Document data overlap and gaps between sources
- [ ] Note price discrepancies between sources
- [ ] Record data quality metrics
- [ ] Identify preferred data source for different time periods

## 5. Data Quality Checks

Performing comprehensive data validation and quality assessment.

In [ ]:
def perform_data_quality_checks(df):
    """
    Comprehensive data quality assessment
    """
    if df is None or df.empty:
        print("❌ No data to check")
        return None
    
    print("🔍 Data Quality Assessment")
    print("="*50)
    
    # Basic info
    print(f"📊 Dataset Shape: {df.shape}")
    print(f"📅 Date Range: {df['date'].min()} to {df['date'].max()}")
    print(f"🔢 Total Days: {(df['date'].max() - df['date'].min()).days} days")
    
    # Missing values
    print("\n🔍 Missing Values:")
    missing_summary = df.isnull().sum()
    for col, missing in missing_summary.items():
        if missing > 0:
            pct = (missing / len(df)) * 100
            print(f"  {col}: {missing} ({pct:.1f}%)")
        else:
            print(f"  {col}: ✅ No missing values")
    
    # Data types
    print("\n📋 Data Types:")
    for col, dtype in df.dtypes.items():
        print(f"  {col}: {dtype}")
    
    # Price statistics
    if 'close' in df.columns:
        print("\n💰 Price Statistics:")
        price_stats = df['close'].describe()
        for stat, value in price_stats.items():
            print(f"  {stat}: ${value:,.2f}")
        
        # Price anomalies
        print("\n⚠️ Price Anomaly Checks:")
        
        # Check for negative prices
        negative_prices = (df['close'] <= 0).sum()
        print(f"  Negative/Zero prices: {negative_prices}")
        
        # Check for extreme price changes (>50% in one day)
        df_sorted = df.sort_values('date')
        price_changes = df_sorted['close'].pct_change().abs()
        extreme_changes = (price_changes > 0.5).sum()
        print(f"  Extreme daily changes (>50%): {extreme_changes}")
        
        if extreme_changes > 0:
            extreme_dates = df_sorted[price_changes > 0.5]['date'].tolist()
            print(f"    Dates with extreme changes: {extreme_dates[:5]}")
    
    # Duplicate checks
    print("\n🔄 Duplicate Checks:")
    date_duplicates = df['date'].duplicated().sum()
    print(f"  Duplicate dates: {date_duplicates}")
    
    # Source distribution
    if 'source' in df.columns:
        print("\n📈 Data Source Distribution:")
        source_counts = df['source'].value_counts()
        for source, count in source_counts.items():
            pct = (count / len(df)) * 100
            print(f"  {source}: {count} records ({pct:.1f}%)")
    
    return {
        'shape': df.shape,
        'missing_values': missing_summary.to_dict(),
        'date_range': (df['date'].min(), df['date'].max()),
        'price_stats': price_stats.to_dict() if 'close' in df.columns else None,
        'anomalies': {
            'negative_prices': negative_prices if 'close' in df.columns else 0,
            'extreme_changes': extreme_changes if 'close' in df.columns else 0
        }
    }

# Run quality checks on combined data
if 'combined_btc_data' in locals() and combined_btc_data is not None:
    quality_report = perform_data_quality_checks(combined_btc_data)
else:
    print("⚠️ No combined data available for quality checks")
    quality_report = None

### Observation 5: Data Quality Assessment

**Critical Findings:**
- [ ] Document completeness percentage by source
- [ ] Note any data quality issues requiring cleaning
- [ ] Record price anomalies and potential causes
- [ ] Assess suitability for trading strategy development

**Next Steps:**
- [ ] Address missing data through interpolation or source switching
- [ ] Implement data cleaning pipeline for anomalies
- [ ] Set up automated data quality monitoring
- [ ] Prepare data for EDA in next notebook

## 6. Export Summary

Final data export and summary for use in subsequent notebooks.

In [ ]:
# Create summary of all data collection efforts
collection_summary = {
    'timestamp': datetime.now().isoformat(),
    'sources_attempted': ['investing.com', 'coinmarketcap', 'yahoo_finance'],
    'sources_successful': [],
    'total_records': 0,
    'date_range': None,
    'files_created': [],
    'quality_report': quality_report
}

# Check which sources were successful
if 'investing_df' in locals() and investing_df is not None:
    collection_summary['sources_successful'].append('investing.com')
    collection_summary['files_created'].append('btc_investing_raw.csv')

if 'yahoo_df' in locals() and yahoo_df is not None:
    collection_summary['sources_successful'].append('yahoo_finance')
    collection_summary['files_created'].append('btc_yahoo_raw.csv')

if 'cmc_data' in locals() and cmc_data is not None:
    collection_summary['sources_successful'].append('coinmarketcap')

if 'combined_btc_data' in locals() and combined_btc_data is not None:
    collection_summary['total_records'] = len(combined_btc_data)
    collection_summary['date_range'] = [
        combined_btc_data['date'].min().isoformat(),
        combined_btc_data['date'].max().isoformat()
    ]
    collection_summary['files_created'].append('btc_combined_raw.csv')

# Save collection summary
import json
with open('../data/collection_summary.json', 'w') as f:
    json.dump(collection_summary, f, indent=2, default=str)

print("📋 Data Collection Summary")
print("="*40)
print(f"✅ Successful sources: {len(collection_summary['sources_successful'])}/{len(collection_summary['sources_attempted'])}")
print(f"📊 Total records collected: {collection_summary['total_records']}")
print(f"📁 Files created: {len(collection_summary['files_created'])}")
print(f"🏁 Ready for EDA: {'✅' if collection_summary['total_records'] > 0 else '❌'}")

if collection_summary['total_records'] > 0:
    print(f"\n📅 Date range: {collection_summary['date_range'][0]} to {collection_summary['date_range'][1]}")
    print("\n🎯 Next step: Run notebook 02_eda_analysis.ipynb for exploratory data analysis")
else:
    print("\n❌ No data collected - check API keys and network connectivity")

### Final Observation: Data Collection Complete

**Summary of Results:**
- [ ] Record final success rate of all data sources
- [ ] Document total data points collected
- [ ] Note any persistent issues with specific sources
- [ ] Confirm readiness for EDA phase

**Key Takeaways for Trading Strategy:**
1. Data reliability ranking by source
2. Recommended fallback strategy for data outages
3. Data freshness considerations for live trading
4. Quality thresholds for strategy execution